In [209]:
import json
import pandas as pd
import numpy as np
import jiwer

In [210]:
result_asr_google_path = "./result_asr_google.txt"
with open(result_asr_google_path, "rb") as ifile:
    lines = ifile.readlines()
    lines = lines[0]
    dict_result_asr_google = json.loads(lines)

In [211]:
lc_path_asr_zalo = '/data4/data_link_async/label_supplier/audio_zalo/2023_03_27/P4d20ec9f81a66bf832b7.wav'
dict_result_asr_google[lc_path_asr_zalo]

''

### READ FROM result_asr_google.txt file

In [212]:
asr_google_df = pd.DataFrame(columns=['file_name', 'lc_path_asr_google', 'predict_google'])

for lc_path, text_predict in dict_result_asr_google.items():
#     print(lc_path, text_predict)
    file_name = lc_path.split("/")[-1]
    df = pd.DataFrame([[file_name, lc_path, text_predict]], columns=['file_name', 'lc_path_asr_google', 'predict_google'])
    asr_google_df = pd.concat([asr_google_df, df])

In [213]:
i = 10
print(asr_google_df['lc_path_asr_google'].values[i])
print(asr_google_df['predict_google'].values[i])

/data4/data_link_async/label_supplier/audio_zalo/2023_03_27/Pbc4cc8f2a5cb4f9516da.wav
ăn hủ tiếu đuôi heo giống như hôm qua hay gì Hay là hủ tiếu thịt hủ tiếu bò viên để thịt viên thôi để bò viên thôi


In [214]:
asr_google_df

,file_name,lc_path_asr_google,predict_google
0,P0562e6df8be661b838f7.wav,/data4/data_link_async/label_supplier/audio_za...,ừ
0,P05236f9c02a5e8fbb1b4.wav,/data4/data_link_async/label_supplier/audio_za...,
0,P4294ad2bc0122a4c7303.wav,/data4/data_link_async/label_supplier/audio_za...,ông cái bà này là khách sĩ cũ mà ở xa em lắm c...
0,P067d77c01af9f0a7a9e8.wav,/data4/data_link_async/label_supplier/audio_za...,ngại đấm cho
0,P235df1e69cdf76812fce.wav,/data4/data_link_async/label_supplier/audio_za...,ở đây làm nữa
...,...,...,...
0,asr-streaming-root-ailab-20230426165555-ea2238...,/data4/data_link_async/label_supplier/audio_za...,mày gọi cơm khô người ta không
0,asr-streaming-root-ailab-20230426164409-4d27d8...,/data4/data_link_async/label_supplier/audio_za...,người mà không lo cho người đàn ông hoàng thôi...
0,asr-streaming-root-ailab-20230426164410-2c605f...,/data4/data_link_async/label_supplier/audio_za...,67 lô 5.000 39 với 18 đánh con hai con nói là ...
0,asr-streaming-root-ailab-20230426164412-e8c4a4...,/data4/data_link_async/label_supplier/audio_za...,Cái gì Tự nhiên nó nói chọc cho con đánh cái đ...


### GET full text from lb1 in database

In [215]:
import re
import copy

def tuple_result_2_dict_result(select_colums, results):
    res_results = []
    
    for result in results:
        assert len(select_colums) == len(result)
        row_dict = {}
        for idx, col_name in enumerate(select_colums):
            row_dict[col_name] = result[idx]
        res_results.append(row_dict)
    
    return res_results


def calculate_wer(gt: str, pred: str):
    if gt:
        return jiwer.wer(gt, pred)
    else:
        return -1

def decode_unicode_escape(string: str = ""):
    replacements = []
    idx = []
    for i in range(len(string)):
        if string[i] == 'u' and i + 4 < len(string) and string[i+1] != 'u':
            text_exist_space = False
            text_exist_u = False
            text_exist_special_char = False
            if 'u' in string[i+1:i+5]:
                text_exist_special_char = True
                
            for j in range(5):
                if string[i+j].isspace() :
                    text_exist_special_char = True
            
            if not text_exist_special_char:
                idx.append(i)
                replacements.append(string[i:i+5])

    for replacement in replacements:
        replace_string = "\\" + replacement
        try:
            decoded_string = replace_string.encode().decode('unicode_escape')
        except Exception as e:
            print(f"ERROR: {e} \nWith text: {replacement}")
            decoded_string = replacement
        string = string.replace(replacement, decoded_string)

    return string

decoded_text = decode_unicode_escape('mi cu00f3 u0103n chu00e1o huynhdai')
print(decoded_text)
decode_unicode_escape('u0103n')

ERROR: 'unicodeescape' codec can't decode bytes in position 0-1: truncated \uXXXX escape 
With text: uynhd
mi có ăn cháo huynhdai


'ăn'

In [216]:
from ailabtools.connection_pool_postgresql import ConnectionPoolPostgreSql

HOSTNAME = 'localhost'
PORT = 5432
DATABASE = 'asr_label_log'
USERNAME = "postgres"
PASSWORD = "postgres"

db = ConnectionPoolPostgreSql(1, 1, HOSTNAME, PORT, USERNAME,
                              PASSWORD, DATABASE, keep_connection=True)

TABLE_NAME = 'asr_zalo'
SELECT_COLUMNS = [
    'id',
    'lc_path',
    'label_url',
    'lb1',
]

sql = f"""
        SELECT {','.join(SELECT_COLUMNS)}
        FROM {TABLE_NAME}
    """
results = db.execute(sql)
column_asr_label_df = ['seed', 'file_name', 'lc_path_asr_zalo', 'label_url', 'full_text', 'predict_kaldi', 'predict_wenet']
asr_label_df = pd.DataFrame(columns=column_asr_label_df)

res_results = tuple_result_2_dict_result(SELECT_COLUMNS, results)
for res in res_results:
    lc_path = res.get("lc_path", "")
    file_name = lc_path.split("/")[-1]
    
    label_url = res.get("label_url", "")
    
    lb1 = res["lb1"]
    
    
    if lb1 == 'EMPTY' or len(lb1) == 0:
        continue
        
    # convert str to dict of lb1
    if isinstance(lb1, str):
        lb1 = json.loads(lb1)
    
    lb1_data = lb1['data']
    full_text = ""
    full_predict_kaldi = ""
    full_predict_wenet = ""
    
    seed = None
    
    for d in lb1_data:
        seed = d.get("seed")
        text = d.get("content", {}).get("tag", {}).get("text", "")
        predict_kaldi = d.get("content", {}).get("extras", {}).get("classify", {}).get("predict_kaldi", "")
        predict_wenet = d.get("content", {}).get("extras", {}).get("classify", {}).get("predict_wenet", "")
        
        # decode
        text = decode_unicode_escape(text)
        predict_kaldi = decode_unicode_escape(predict_kaldi)
        predict_wenet = decode_unicode_escape(predict_wenet)
        
        
        # replace None value
        predict_kaldi = predict_kaldi.replace("None", "")
        predict_wenet = predict_wenet.replace("None", "")
        if not (text == 'zzz' or text == 'zzzz'):
            full_text = full_text + " " + text.strip()
        
        full_predict_kaldi = full_predict_kaldi + " " + predict_kaldi.strip()
        full_predict_wenet = full_predict_wenet + " " + predict_wenet.strip()
        
    full_text = full_text.strip()
    full_predict_kaldi = full_predict_kaldi.strip()
    full_predict_wenet = full_predict_wenet.strip()
    
    df = pd.DataFrame([[seed, file_name, lc_path, label_url, full_text, full_predict_kaldi, full_predict_wenet]], columns=column_asr_label_df)
    asr_label_df = pd.concat([asr_label_df, df])

# asr_label_df.to_csv("asr_label_full_text.csv")
asr_label_df

ERROR: 'unicodeescape' codec can't decode bytes in position 0-1: truncated \uXXXX escape 
With text: undai


/home/thanhnd13/miniconda3/envs/py310/lib/python3.10/site-packages/ailabtools/zlogger.py:16: UserWarning: This instance is already created so re-use initialized parameters!
  warnings.warn(


ERROR: 'unicodeescape' codec can't decode bytes in position 0-1: truncated \uXXXX escape 
With text: uto(a
ERROR: 'unicodeescape' codec can't decode bytes in position 0-1: truncated \uXXXX escape 
With text: uynda
ERROR: 'unicodeescape' codec can't decode bytes in position 0-1: truncated \uXXXX escape 
With text: uynhd
ERROR: 'unicodeescape' codec can't decode bytes in position 0-1: truncated \uXXXX escape 
With text: untai
ERROR: 'unicodeescape' codec can't decode bytes in position 0-1: truncated \uXXXX escape 
With text: utrav
ERROR: 'unicodeescape' codec can't decode bytes in position 0-1: truncated \uXXXX escape 
With text: utrav


,seed,file_name,lc_path_asr_zalo,label_url,full_text,predict_kaldi,predict_wenet
0,1470,P00471cf671cf9b91c2de.wav,/data4/data_link_async/label_supplier/audio_za...,https://api.zalo.ai/ailab_video/label_supplier...,mi có ăn chay trưòng cái tâm mi mới mới mới tị...,mi có ăn cháo ừ cái tâm ý mới tận lệ thôi,
0,1471,P00b3f4089931736f2a20.wav,/data4/data_link_async/label_supplier/audio_za...,https://api.zalo.ai/ailab_video/label_supplier...,chạy không nói nhanh nhe xe vô gần tới rồi nữa...,,
0,1472,P00c3fc7991407b1e2251.wav,/data4/data_link_async/label_supplier/audio_za...,https://api.zalo.ai/ailab_video/label_supplier...,đụ mẹ cái dây sạc bị hư rồi do củ sạc hả ta,,
0,1473,P00f65c483171db2f8260.wav,/data4/data_link_async/label_supplier/audio_za...,https://api.zalo.ai/ailab_video/label_supplier...,hello (hé lô) ừm chị coi lại nè cái toa này tr...,,
0,1474,P014bc7f5aacc409219dd.wav,/data4/data_link_async/label_supplier/audio_za...,https://api.zalo.ai/ailab_video/label_supplier...,kê ô kê anh thế thì em bao luật hai triệu cho ...,,
...,...,...,...,...,...,...,...
0,4991,asr-streaming-root-ailab-20230426164330-77152b...,/data4/data_link_async/label_supplier/audio_za...,https://api.zalo.ai/ailab_video/label_supplier...,xem còn xuồng nào nữa gì mấy xuồng nó lấy được...,,
0,4996,asr-streaming-root-ailab-20230426164335-be7d63...,/data4/data_link_async/label_supplier/audio_za...,https://api.zalo.ai/ailab_video/label_supplier...,em a tối đi tối báo anh khi nào em họp xong đi...,,
0,4997,asr-streaming-root-ailab-20230426164337-22ed7d...,/data4/data_link_async/label_supplier/audio_za...,https://api.zalo.ai/ailab_video/label_supplier...,ba hết tiền rồi cho ba vay triệu rưỡi đi,,
0,4998,asr-streaming-root-ailab-20230426164338-9210f7...,/data4/data_link_async/label_supplier/audio_za...,https://api.zalo.ai/ailab_video/label_supplier...,,,


In [217]:
asr_label_df = asr_label_df.set_index('file_name')
asr_google_df = asr_google_df.set_index('file_name')

In [218]:
df_concat = pd.concat([asr_label_df, asr_google_df], axis=1)
df_concat = df_concat.reset_index()
df_concat.index.name = 'id'


In [219]:
df_concat['full_text'] = df_concat['full_text'].fillna("")
df_concat['predict_kaldi'] = df_concat['predict_kaldi'].fillna("")
df_concat['predict_wenet'] = df_concat['predict_wenet'].fillna("")
df_concat['predict_google'] = df_concat['predict_google'].fillna("")

### Handle WER

In [ ]:
df_concat['wer_google'] = df_concat.apply(lambda row: str(calculate_wer(row['full_text'], row['predict_google'])), axis=1)
df_concat['wer_kaldi'] = df_concat.apply(lambda row: str(calculate_wer(row['full_text'], row['predict_kaldi'])), axis=1)
df_concat['wer_wenet'] = df_concat.apply(lambda row: str(calculate_wer(row['full_text'], row['predict_wenet'])), axis=1)

# for idx, row in df_concat.iterrows():
# #     print('---')
# #     row = dict(row)
 
#     full_text = row['full_text'].strip()
#     predict_google = row['predict_google'].strip()
#     predict_kaldi = row['predict_kaldi'].strip()
#     predict_wenet = row['predict_wenet'].strip()
    
#     wer_google = calculate_wer(full_text, predict_google)
#     wer_kaldi = calculate_wer(full_text, predict_kaldi)
#     wer_wenet = calculate_wer(full_text, predict_wenet)

    
#     df_concat.at[idx, 'wer_google'] = wer_google
#     df_concat.at[idx, 'wer_kaldi'] = wer_kaldi
#     df_concat.at[idx, 'wer_wenet'] = wer_wenet
    
df_asr_benchmark = df_concat[['seed', 'label_url', 'full_text',
       'predict_kaldi', 'predict_wenet', 
       'predict_google', 'wer_google', 'wer_kaldi', 'wer_wenet']]
df_asr_benchmark.to_csv("asr_benchmark.csv")
df_asr_benchmark

,seed,label_url,full_text,predict_kaldi,predict_wenet,predict_google,wer_google,wer_kaldi,wer_wenet
id,,,,,,,,,
0,1470,https://api.zalo.ai/ailab_video/label_supplier...,mi có ăn chay trưòng cái tâm mi mới mới mới tị...,mi có ăn cháo ừ cái tâm ý mới tận lệ thôi,,mày gọi anh tra từ có tầm mấy,1.0,0.5,1.0
1,1471,https://api.zalo.ai/ailab_video/label_supplier...,chạy không nói nhanh nhe xe vô gần tới rồi nữa...,,,chạy hâm nóng nhanh nha xe vào gần tới rồi nửa...,0.29411764705882354,1.0,1.0
2,1472,https://api.zalo.ai/ailab_video/label_supplier...,đụ mẹ cái dây sạc bị hư rồi do củ sạc hả ta,,,,1.0,1.0,1.0
3,1473,https://api.zalo.ai/ailab_video/label_supplier...,hello (hé lô) ừm chị coi lại nè cái toa này tr...,,,,1.0,1.0,1.0
4,1474,https://api.zalo.ai/ailab_video/label_supplier...,kê ô kê anh thế thì em bao luật hai triệu cho ...,,,,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...
3797,NaN,NaN,,,,mày gọi cơm khô người ta không,-1,-1,-1
3798,NaN,NaN,,,,người mà không lo cho người đàn ông hoàng thôi...,-1,-1,-1
3799,NaN,NaN,,,,67 lô 5.000 39 với 18 đánh con hai con nói là ...,-1,-1,-1


In [ ]:
df_concat.columns

Index(['file_name', 'seed', 'lc_path_asr_zalo', 'label_url', 'full_text',
       'predict_kaldi', 'predict_wenet', 'lc_path_asr_google',
       'predict_google', 'wer_google', 'wer_kaldi', 'wer_wenet'],
      dtype='object')

In [258]:
calculate_wer("12", "")

1.0